In [ ]:
import os
import numpy as np
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torch
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings


In [ ]:
# Configuration variables
CHUNK_SIZE = 300
CHUNK_OVERLAP = 50

# Step 1 : Load the CSV file
csv_loader = CSVLoader(file_path='3_building_simple_rag_pipeline\tesla_motors_data.csv')
documents = csv_loader.load()

In [ ]:
# Step2 : Split the text into chunks
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
texts = text_splitter.split_documents(documents)

In [ ]:
#Step 3:  Create embeddings
embedding_model = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings = embedding_model.embed_documents(texts)

In [ ]:
#Step 4:  Store embeddings in FAISS

def create_vectorstore(texts, embeddings):
    vectorstore = FAISS.from_documents(texts, embeddings)
    return vectorstore
vector_store =create_vectorstore(texts, embeddings)

In [ ]:
# Create a retrieval system
retriever = vector_store.as_retriever()

In [ ]:
# Set up the LLM
llama_model = LlamaForCausalLM.from_pretrained('path/to/llama/model')
llama_tokenizer = LlamaTokenizer.from_pretrained('path/to/llama/tokenizer')
llm = HuggingFacePipeline(model=llama_model, tokenizer=llama_tokenizer, device=0 if torch.cuda.is_available() else -1)

In [ ]:
# Create a chatbot function
def chatbot(query: str) -> str:
    # Retrieve relevant documents
    relevant_docs = retriever.retrieve(query)
    
    # Format the prompt
    prompt_template = PromptTemplate(
        input_variables=["query", "documents"],
        template="""
        You are an expert in information retrieval and natural language processing.
        Given the following query and a list of relevant documents, generate a comprehensive response.

        Query: {query}

        Documents:
        {documents}

        Response:
        """
    )
    formatted_prompt = prompt_template.format(query=query, documents="\n".join([doc.page_content for doc in relevant_docs]))
    
    # Generate response using Llama
    response = llm(formatted_prompt)
    
    return response

# Example usage
if __name__ == "__main__":
    while True:
        user_query = input("Enter your question (or 'quit' to exit): ").strip()
        if user_query.lower() == 'quit':
            break
        response = chatbot(user_query)
        print(f"Response: {response}")